In [1]:
"""
Given the following dataset, classify customer value based on order frequency. 

You can assign them a score of 1-3, with 1 being least valuable and 3 being most valuable. 

For example, a customer who has made 10 purchases is more valuable than one that has made 
2 purchases across the same time period.

For more information regarding customer lifetime value you can read 
this post: https://www.shopify.com/blog/customer-lifetime-value

The solution will be written in Python for premium users. 

Note that this question is a part of a series of questions that will build off of each other.
"""
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
plt.style.use('fivethirtyeight')

data = pd.read_csv('./data/Online_Retail.csv')
data.sample(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
41821,539958,84968F,SET OF 16 VINTAGE SKY BLUE CUTLERY,3,12/23/10 13:26,8.47,NaN,United Kingdom
50314,540551,21222,SET/4 BADGES BEETLES,1,1/10/11 9:43,0.85,NaN,United Kingdom
24972,538351,21452,TOADSTOOL MONEY BOX,3,12/10/10 15:17,6.77,NaN,United Kingdom
899,536488,21367,MIRRORED WALL ART GENTS,1,12/1/10 12:31,2.55,17897.0,United Kingdom
5980,536876,22865,HAND WARMER OWL DESIGN,6,12/3/10 11:36,4.21,NaN,United Kingdom
63691,541590,21537,RED RETROSPOT PUDDING BOWL,4,1/19/11 14:50,4.25,13310.0,United Kingdom
380,536401,22428,ENAMEL FIRE BUCKET CREAM,2,12/1/10 11:21,6.95,15862.0,United Kingdom
23204,538184,22492,MINI PAINT SET VINTAGE,36,12/10/10 10:21,0.65,17880.0,United Kingdom
9978,537224,84881,BLUE WIRE SPIRAL CANDLE HOLDER,2,12/5/10 16:24,6.35,13174.0,United Kingdom
38712,539595,48184,DOORMAT ENGLISH ROSE,1,12/20/10 13:43,14.43,NaN,United Kingdom


In [2]:
data.isna().sum()

InvoiceNo          0
StockCode          0
Description      166
Quantity           0
InvoiceDate        0
UnitPrice          0
CustomerID     25281
Country            0
dtype: int64

In [3]:
data.InvoiceDate.min(), data.InvoiceDate.max()

('1/10/11 10:04', '12/9/10 9:49')

In [4]:
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])

In [5]:
data.InvoiceDate.min(), data.InvoiceDate.max()

(Timestamp('2010-12-01 08:26:00'), Timestamp('2011-01-20 18:01:00'))

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65499 entries, 0 to 65498
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   InvoiceNo    65499 non-null  object        
 1   StockCode    65499 non-null  object        
 2   Description  65333 non-null  object        
 3   Quantity     65499 non-null  int64         
 4   InvoiceDate  65499 non-null  datetime64[ns]
 5   UnitPrice    65499 non-null  float64       
 6   CustomerID   40218 non-null  float64       
 7   Country      65499 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 4.0+ MB


In [7]:
data['Spent'] = data['Quantity'] * data['UnitPrice']
data.sample(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Spent
61237,541424,22403,MAGNETS PACK OF 4 VINTAGE LABELS,4,2011-01-17 17:57:00,0.83,NaN,United Kingdom,3.32
11175,537240,22075,6 RIBBONS ELEGANT CHRISTMAS,1,2010-12-06 10:08:00,3.36,NaN,United Kingdom,3.36
9469,537202,20969,RED FLORAL FELTCRAFT SHOULDER BAG,1,2010-12-05 14:25:00,3.75,14506.0,United Kingdom,3.75
47986,540458,22149,FELTCRAFT 6 FLOWER FRIENDS,6,2011-01-07 12:28:00,2.10,12501.0,Germany,12.60
10296,537237,21224,SET/4 SKULL BADGES,1,2010-12-06 09:58:00,2.51,NaN,United Kingdom,2.51
28275,538634,22601,CHRISTMAS RETROSPOT ANGEL WOOD,8,2010-12-13 13:28:00,0.85,14085.0,United Kingdom,6.80
41199,539760,21738,COSY SLIPPER SHOES SMALL RED,3,2010-12-21 17:16:00,2.95,17400.0,United Kingdom,8.85
44527,540179,22624,IVORY KITCHEN SCALES,1,2011-01-05 12:54:00,8.50,16033.0,United Kingdom,8.50
34762,539434,84970S,HANGING HEART ZINC T-LIGHT HOLDER,3,2010-12-17 14:41:00,2.13,NaN,United Kingdom,6.39
15510,537638,37450,CERAMIC CAKE BOWL + HANGING CAKES,3,2010-12-07 15:28:00,5.91,NaN,United Kingdom,17.73


In [8]:
data[~data['CustomerID'].isna()].shape

(40218, 9)

In [9]:
data = data[~data['CustomerID'].isna()]
data.isna().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
Spent          0
dtype: int64

In [10]:
data.shape

(40218, 9)

In [11]:
cust_ids_lst = list(data.CustomerID.unique())
cust_ids_lst

[17850.0,
 13047.0,
 12583.0,
 13748.0,
 15100.0,
 15291.0,
 14688.0,
 17809.0,
 15311.0,
 14527.0,
 16098.0,
 18074.0,
 17420.0,
 16029.0,
 16250.0,
 12431.0,
 17511.0,
 17548.0,
 13705.0,
 13747.0,
 13408.0,
 13767.0,
 17924.0,
 13448.0,
 15862.0,
 15513.0,
 12791.0,
 16218.0,
 14045.0,
 14307.0,
 17908.0,
 17920.0,
 12838.0,
 13255.0,
 16583.0,
 18085.0,
 13758.0,
 13694.0,
 15983.0,
 14849.0,
 17968.0,
 16210.0,
 17897.0,
 17377.0,
 16552.0,
 17181.0,
 17951.0,
 14729.0,
 12748.0,
 15012.0,
 12868.0,
 17572.0,
 14078.0,
 14001.0,
 12662.0,
 15525.0,
 14237.0,
 17905.0,
 15485.0,
 12433.0,
 16955.0,
 15350.0,
 15605.0,
 18144.0,
 15922.0,
 14594.0,
 15165.0,
 14911.0,
 16456.0,
 17841.0,
 12472.0,
 17346.0,
 17643.0,
 17873.0,
 13093.0,
 12921.0,
 13468.0,
 17760.0,
 16928.0,
 16048.0,
 16274.0,
 14496.0,
 14696.0,
 16539.0,
 17025.0,
 13777.0,
 17690.0,
 12947.0,
 17460.0,
 18229.0,
 14142.0,
 17069.0,
 13065.0,
 14606.0,
 16835.0,
 15235.0,
 13576.0,
 18011.0,
 13090.0,
 15694.0,


In [12]:
spent_stats_dict = data.groupby('CustomerID')['Spent'].sum().describe().to_dict()
spent_stats_dict

{'count': 1204.0,
 'mean': 729.8498006644518,
 'std': 1915.0425086636212,
 'min': -1192.2000000000003,
 '25%': 177.2325,
 '50%': 324.31,
 '75%': 662.3525,
 'max': 27834.61}

In [13]:
spent_dict = data.groupby('CustomerID')['Spent'].sum().to_dict()
spent_dict

{12346.0: 0.0,
 12347.0: 711.79,
 12348.0: 892.8000000000001,
 12356.0: 2271.6200000000003,
 12359.0: 547.5,
 12370.0: 1868.02,
 12377.0: 1001.5199999999996,
 12383.0: 1240.6299999999999,
 12386.0: 401.90000000000003,
 12388.0: 431.30000000000007,
 12393.0: 639.5,
 12395.0: 676.9699999999999,
 12402.0: 225.6,
 12413.0: 415.30000000000007,
 12415.0: 7092.979999999998,
 12417.0: 259.39,
 12422.0: 238.5,
 12423.0: 237.93,
 12427.0: 246.55000000000007,
 12429.0: 1281.5000000000002,
 12431.0: 773.9499999999999,
 12433.0: 3787.1199999999994,
 12434.0: -27.749999999999996,
 12437.0: 392.7,
 12441.0: 173.55,
 12451.0: 2919.5099999999998,
 12471.0: 4189.16,
 12472.0: 1509.0000000000007,
 12474.0: 215.4,
 12476.0: -12.45,
 12480.0: 816.2500000000002,
 12481.0: 1394.4300000000003,
 12483.0: 507.55999999999995,
 12484.0: 1138.6700000000005,
 12489.0: 334.92999999999995,
 12490.0: 864.2199999999999,
 12494.0: 37.80000000000001,
 12500.0: 344.93999999999994,
 12501.0: 2169.3899999999994,
 12503.0: -

In [14]:
def rating_func(customer, dict_, stats_dict_):
    if dict_[customer] < stats_dict_['25%']:
        return 1
    elif dict_[customer] < stats_dict_['50%']:
        return 2
    elif dict_[customer] < stats_dict_['75%']:
        return 3
    else:
        return 4

rating_func(12377.0, spent_dict, spent_stats_dict)

4

In [15]:
spent_dict[12377.0]

1001.5199999999996

In [16]:
spent_ratings = []
for ids in cust_ids_lst:
    spent_ratings.append(rating_func(ids, spent_dict, spent_stats_dict))
spent_ratings

[4,
 3,
 4,
 2,
 3,
 4,
 4,
 4,
 4,
 4,
 3,
 3,
 1,
 4,
 2,
 4,
 4,
 1,
 2,
 1,
 4,
 4,
 2,
 4,
 3,
 4,
 2,
 3,
 4,
 4,
 2,
 4,
 3,
 2,
 2,
 4,
 4,
 4,
 4,
 3,
 2,
 4,
 3,
 3,
 1,
 1,
 3,
 2,
 4,
 2,
 3,
 1,
 1,
 2,
 2,
 2,
 1,
 3,
 4,
 4,
 3,
 1,
 2,
 3,
 3,
 2,
 3,
 4,
 4,
 4,
 4,
 2,
 1,
 2,
 4,
 4,
 4,
 1,
 1,
 2,
 3,
 2,
 3,
 4,
 1,
 4,
 4,
 3,
 1,
 4,
 2,
 3,
 2,
 4,
 1,
 4,
 4,
 1,
 4,
 4,
 3,
 4,
 3,
 2,
 1,
 2,
 2,
 2,
 2,
 4,
 4,
 4,
 3,
 4,
 4,
 4,
 1,
 2,
 2,
 3,
 4,
 3,
 1,
 4,
 2,
 2,
 1,
 3,
 1,
 1,
 4,
 1,
 2,
 2,
 3,
 2,
 3,
 3,
 1,
 4,
 2,
 4,
 3,
 2,
 2,
 3,
 3,
 2,
 1,
 2,
 3,
 2,
 4,
 2,
 1,
 3,
 2,
 4,
 4,
 1,
 2,
 1,
 3,
 4,
 1,
 4,
 3,
 2,
 4,
 3,
 3,
 3,
 4,
 3,
 4,
 1,
 3,
 4,
 2,
 1,
 2,
 4,
 1,
 2,
 3,
 4,
 3,
 1,
 4,
 2,
 1,
 2,
 4,
 1,
 3,
 1,
 2,
 4,
 1,
 4,
 1,
 3,
 2,
 2,
 3,
 3,
 2,
 3,
 1,
 4,
 1,
 1,
 1,
 1,
 4,
 1,
 4,
 4,
 2,
 2,
 1,
 2,
 1,
 4,
 3,
 4,
 3,
 4,
 4,
 3,
 2,
 4,
 4,
 4,
 2,
 4,
 4,
 3,
 3,
 4,
 4,
 3,
 1,
 3,
 3,
 3,
 4,
 1,
 2,
 3,


In [17]:
date_stats_dict = data.groupby('CustomerID')['InvoiceDate'].max().describe(datetime_is_numeric=True).to_dict()
date_stats_dict

{'count': 1204,
 'mean': Timestamp('2010-12-25 08:58:36.578073088'),
 'min': Timestamp('2010-12-01 09:00:00'),
 '25%': Timestamp('2010-12-09 14:44:45'),
 '50%': Timestamp('2010-12-18 14:43:00'),
 '75%': Timestamp('2011-01-11 11:49:15'),
 'max': Timestamp('2011-01-20 17:06:00')}

In [18]:
date_dict = data.groupby('CustomerID')['InvoiceDate'].max().to_dict()
date_dict

{12346.0: Timestamp('2011-01-18 10:17:00'),
 12347.0: Timestamp('2010-12-07 14:57:00'),
 12348.0: Timestamp('2010-12-16 19:09:00'),
 12356.0: Timestamp('2011-01-18 09:50:00'),
 12359.0: Timestamp('2011-01-12 12:43:00'),
 12370.0: Timestamp('2010-12-17 09:38:00'),
 12377.0: Timestamp('2010-12-20 09:37:00'),
 12383.0: Timestamp('2011-01-19 14:36:00'),
 12386.0: Timestamp('2011-01-06 12:37:00'),
 12388.0: Timestamp('2011-01-17 11:12:00'),
 12393.0: Timestamp('2011-01-14 11:36:00'),
 12395.0: Timestamp('2011-01-05 11:21:00'),
 12402.0: Timestamp('2011-01-20 13:49:00'),
 12413.0: Timestamp('2011-01-12 09:52:00'),
 12415.0: Timestamp('2011-01-10 09:58:00'),
 12417.0: Timestamp('2011-01-06 17:57:00'),
 12422.0: Timestamp('2011-01-19 09:13:00'),
 12423.0: Timestamp('2010-12-21 10:54:00'),
 12427.0: Timestamp('2010-12-23 10:20:00'),
 12429.0: Timestamp('2010-12-09 12:05:00'),
 12431.0: Timestamp('2010-12-17 14:10:00'),
 12433.0: Timestamp('2010-12-08 16:49:00'),
 12434.0: Timestamp('2010-12-14 

In [19]:
date_ratings = []
for ids in cust_ids_lst:
    date_ratings.append(rating_func(ids, date_dict, date_stats_dict))
date_ratings

[1,
 3,
 3,
 1,
 4,
 4,
 4,
 1,
 4,
 4,
 1,
 1,
 1,
 4,
 1,
 2,
 3,
 1,
 1,
 1,
 3,
 3,
 3,
 2,
 1,
 4,
 1,
 1,
 3,
 4,
 1,
 1,
 1,
 1,
 1,
 4,
 3,
 4,
 2,
 2,
 1,
 2,
 1,
 2,
 1,
 1,
 3,
 1,
 4,
 1,
 4,
 1,
 1,
 1,
 1,
 1,
 1,
 3,
 3,
 1,
 3,
 1,
 1,
 2,
 1,
 1,
 1,
 4,
 1,
 4,
 1,
 2,
 1,
 1,
 4,
 3,
 3,
 1,
 1,
 1,
 1,
 1,
 4,
 2,
 1,
 4,
 4,
 3,
 1,
 4,
 1,
 3,
 1,
 4,
 1,
 3,
 3,
 1,
 4,
 4,
 2,
 3,
 3,
 1,
 1,
 1,
 1,
 3,
 1,
 1,
 2,
 4,
 1,
 4,
 3,
 2,
 2,
 1,
 1,
 2,
 4,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 3,
 1,
 1,
 1,
 1,
 1,
 1,
 4,
 4,
 3,
 1,
 4,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 4,
 1,
 1,
 1,
 2,
 1,
 3,
 1,
 4,
 4,
 1,
 1,
 3,
 1,
 4,
 4,
 3,
 2,
 2,
 2,
 1,
 2,
 2,
 4,
 1,
 4,
 1,
 2,
 1,
 1,
 1,
 1,
 4,
 1,
 1,
 2,
 3,
 2,
 1,
 4,
 2,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 3,
 1,
 3,
 1,
 3,
 1,
 1,
 4,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 2,
 1,
 1,
 4,
 4,
 1,
 3,
 1,
 1,
 1,
 3,
 1,
 3,
 3,
 1,
 3,
 3,
 2,
 3,
 3,
 4,
 1,
 4,
 3,
 2,
 4,
 2,
 4,
 3,
 1,
 1,
 1,
 3,
 3,
 1,
 4,
 1,


In [20]:
count_stats_dict = data.groupby('CustomerID')['InvoiceDate'].count().describe().to_dict()
count_stats_dict

{'count': 1204.0,
 'mean': 33.403654485049834,
 'std': 46.151128948007496,
 'min': 1.0,
 '25%': 9.0,
 '50%': 19.0,
 '75%': 42.0,
 'max': 695.0}

In [21]:
count_dict = data.groupby('CustomerID')['InvoiceDate'].count().to_dict()
count_dict

{12346.0: 2,
 12347.0: 31,
 12348.0: 17,
 12356.0: 36,
 12359.0: 16,
 12370.0: 91,
 12377.0: 43,
 12383.0: 69,
 12386.0: 10,
 12388.0: 19,
 12393.0: 22,
 12395.0: 32,
 12402.0: 11,
 12413.0: 26,
 12415.0: 47,
 12417.0: 14,
 12422.0: 13,
 12423.0: 16,
 12427.0: 13,
 12429.0: 20,
 12431.0: 24,
 12433.0: 147,
 12434.0: 3,
 12437.0: 7,
 12441.0: 11,
 12451.0: 101,
 12471.0: 109,
 12472.0: 84,
 12474.0: 14,
 12476.0: 6,
 12480.0: 42,
 12481.0: 46,
 12483.0: 14,
 12484.0: 98,
 12489.0: 15,
 12490.0: 43,
 12494.0: 7,
 12500.0: 5,
 12501.0: 149,
 12503.0: 1,
 12510.0: 29,
 12515.0: 16,
 12523.0: 7,
 12524.0: 16,
 12527.0: 10,
 12530.0: 13,
 12539.0: 93,
 12540.0: 52,
 12551.0: 10,
 12553.0: 38,
 12557.0: 9,
 12567.0: 83,
 12577.0: 67,
 12578.0: 71,
 12583.0: 42,
 12585.0: 74,
 12586.0: 1,
 12600.0: 7,
 12601.0: 9,
 12605.0: 3,
 12621.0: 33,
 12626.0: 53,
 12637.0: 72,
 12643.0: 2,
 12647.0: 53,
 12649.0: 2,
 12651.0: 9,
 12652.0: 48,
 12662.0: 15,
 12665.0: 4,
 12666.0: 2,
 12668.0: 24,
 12672

In [22]:
count_ratings = []
for ids in cust_ids_lst:
    count_ratings.append(rating_func(ids, count_dict, count_stats_dict))
count_ratings

[4,
 2,
 4,
 1,
 1,
 3,
 3,
 2,
 4,
 4,
 2,
 2,
 1,
 3,
 2,
 3,
 4,
 1,
 2,
 1,
 4,
 4,
 1,
 3,
 4,
 3,
 1,
 3,
 1,
 4,
 4,
 4,
 4,
 1,
 2,
 3,
 3,
 4,
 4,
 3,
 4,
 3,
 4,
 4,
 1,
 1,
 2,
 4,
 4,
 4,
 3,
 1,
 2,
 2,
 2,
 4,
 2,
 3,
 4,
 4,
 2,
 1,
 2,
 1,
 2,
 3,
 3,
 4,
 2,
 4,
 4,
 4,
 1,
 2,
 4,
 4,
 4,
 2,
 1,
 1,
 4,
 1,
 3,
 3,
 1,
 4,
 4,
 3,
 1,
 2,
 3,
 3,
 2,
 4,
 1,
 4,
 3,
 3,
 2,
 3,
 3,
 4,
 4,
 2,
 1,
 2,
 2,
 2,
 1,
 2,
 4,
 4,
 2,
 4,
 4,
 4,
 1,
 2,
 3,
 3,
 4,
 3,
 1,
 4,
 2,
 3,
 2,
 2,
 1,
 1,
 3,
 3,
 2,
 3,
 2,
 2,
 3,
 3,
 1,
 2,
 2,
 4,
 3,
 4,
 3,
 2,
 2,
 2,
 2,
 3,
 4,
 4,
 1,
 3,
 1,
 3,
 3,
 3,
 4,
 1,
 2,
 2,
 3,
 4,
 2,
 4,
 4,
 2,
 4,
 3,
 3,
 3,
 4,
 3,
 3,
 1,
 2,
 1,
 2,
 1,
 3,
 4,
 1,
 2,
 4,
 4,
 3,
 1,
 4,
 3,
 1,
 2,
 1,
 1,
 3,
 1,
 1,
 4,
 1,
 3,
 2,
 3,
 2,
 1,
 4,
 4,
 2,
 1,
 1,
 3,
 1,
 1,
 1,
 1,
 4,
 2,
 3,
 4,
 2,
 2,
 4,
 4,
 1,
 4,
 4,
 2,
 4,
 4,
 3,
 3,
 1,
 2,
 1,
 3,
 2,
 4,
 4,
 4,
 4,
 2,
 3,
 3,
 1,
 4,
 4,
 3,
 2,
 3,
 2,
 3,


In [23]:
ratings_dict = {'CustomerID': cust_ids_lst
                , 'SpentRating': spent_ratings 
                , 'DateRating': date_ratings
                , 'CountRating': count_ratings
               }

data_ratings = pd.DataFrame(ratings_dict)
data_ratings.sample(10)

,CustomerID,SpentRating,DateRating,CountRating
446,12839.0,4,2,3
737,12434.0,1,2,1
582,16519.0,1,1,1
73,17873.0,2,1,2
55,15525.0,2,1,4
212,15240.0,1,1,1
296,16393.0,2,2,3
523,13531.0,2,1,2
281,13506.0,2,1,2
908,14646.0,4,4,4


In [24]:
data_ratings['SumRating'] = data_ratings['SpentRating'] + data_ratings['DateRating'] + data_ratings['CountRating']
data_ratings.sample(10)

,CustomerID,SpentRating,DateRating,CountRating,SumRating
887,15204.0,2,2,2,6
90,14142.0,2,1,3,6
444,16950.0,1,1,1,3
145,14466.0,3,1,2,6
122,15070.0,1,1,1,3
850,16560.0,3,2,3,8
725,13324.0,3,2,1,6
321,13579.0,1,1,2,4
717,17890.0,3,2,4,9
950,16282.0,1,3,2,6


In [25]:
data_ratings.SumRating.value_counts()

8     174
7     164
6     146
5     129
11    123
9     121
10    104
4     101
12     77
3      65
Name: SumRating, dtype: int64

In [26]:
3, 4, 5, 6, 7, 8, 9, 10, 11, 12

(3, 4, 5, 6, 7, 8, 9, 10, 11, 12)

In [27]:
def three_rating(num):
    if num < 6:
        return 1
    elif num < 10:
        return 2
    else:
        return 3

three_rating(10)
    

3

In [28]:
data_ratings['ThreeRating'] = data_ratings['SumRating'].apply(three_rating)
data_ratings.sample(10)

,CustomerID,SpentRating,DateRating,CountRating,SumRating,ThreeRating
25,15513.0,4,4,3,11,3
151,14449.0,2,1,4,7,2
703,17990.0,2,3,2,7,2
802,15062.0,2,2,2,6,2
831,14414.0,2,2,1,5,1
537,17655.0,3,2,2,7,2
930,13487.0,2,3,2,7,2
771,14150.0,2,2,2,6,2
14,16250.0,2,1,2,5,1
1035,14755.0,3,3,3,9,2


In [29]:
data_ratings['ThreeRating'].value_counts(sort=False)

1    295
2    605
3    304
Name: ThreeRating, dtype: int64

In [33]:
best_cust = list(data_ratings[data_ratings['ThreeRating'] == 3]['CustomerID'].values)
med_cust = list(data_ratings[data_ratings['ThreeRating'] == 2]['CustomerID'].values)
ok_cust = list(data_ratings[data_ratings['ThreeRating'] == 2]['CustomerID'].values)